In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q htt  https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [41.5 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ge

In [ ]:
from pyspark.sql.functions import explode,col
from pyspark.sql.types import *

# **Read the Data**

**Spark JSON data source API provides the multiline option to read records from multiple lines.
By default, spark considers every record in a JSON file as a fully qualified record in a single line hence, we need to use the multiline option to process JSON from multiple lines.**

In [ ]:
df=spark.read.option("multiline","true").json('/content/Py-SparkcodetoFlattenJSON.json')
df.show()

+--------------------+---------+
|               feeds|totalFeed|
+--------------------+---------+
|[[0, 0, 2020-01-0...|      125|
+--------------------+---------+



# **Check the schema of the Data**

In [ ]:
df.printSchema()

root
 |-- feeds: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- code: long (nullable = true)
 |    |    |-- commentCount: long (nullable = true)
 |    |    |-- createdAt: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- feedsComment: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- imagePaths: string (nullable = true)
 |    |    |-- images: string (nullable = true)
 |    |    |-- isdeleted: boolean (nullable = true)
 |    |    |-- lat: long (nullable = true)
 |    |    |-- likeDislike: struct (nullable = true)
 |    |    |    |-- dislikes: long (nullable = true)
 |    |    |    |-- likes: long (nullable = true)
 |    |    |    |-- userAction: long (nullable = true)
 |    |    |-- lng: long (nullable = true)
 |    |    |-- location: string (nullable = true)
 |    |    |-- mediatype: long (nullable = true)
 |    |    |-- msg: string (nullable = true)
 |    |    |-- multiMe

In [ ]:
df.columns

['feeds', 'totalFeed']

In [ ]:
klsa=df.columns;

In [ ]:
df.dtypes

[('feeds',
  'array<struct<code:bigint,commentCount:bigint,createdAt:string,description:string,feedsComment:string,id:bigint,imagePaths:string,images:string,isdeleted:boolean,lat:bigint,likeDislike:struct<dislikes:bigint,likes:bigint,userAction:bigint>,lng:bigint,location:string,mediatype:bigint,msg:string,multiMedia:array<struct<createAt:string,description:string,id:bigint,likeCount:bigint,mediatype:bigint,name:string,place:string,url:string>>,name:string,profilePicture:string,title:string,userId:bigint,videoUrl:string>>'),
 ('totalFeed', 'bigint')]

# **Initialization**

In [ ]:
array_cols=[];
structure_cols=[];

In [ ]:
temp=0;

In [ ]:
data_types=df.dtypes;
for c in data_types:
  if c[1][0:5] == "array":
    array_cols.append(c[0]);
  elif c[1][0:6] == "struct":
    structure_cols.append(c[0]);

# **Main Loop Execution**

In [ ]:
while temp!=1:
  #Number of columns present in the current DataFrame which are Array
  length_array_cols=len(array_cols); 
  #Number of columns present in the current DataFrame which are Structures
  length_structure_cols=len(structure_cols); 
  #Explode all the Arrays
  for i in range(length_array_cols):
    df=df.withColumn("Expanded_"+array_cols[i],explode(df[array_cols[i]])).drop(array_cols[i]);
  #Take all the Structure columns & flat it
  for i in range(length_structure_cols):
    dfstruct=df.schema[structure_cols[i]].dataType.fields;#Extracting the fields of the structure columns
    number_of_fields=len(dfstruct);
    for mj in range(number_of_fields):#Iterate through all the fields & store those in new column in original DataFrame
      df=df.withColumn(structure_cols[i]+"_"+dfstruct[mj].name,df[structure_cols[i]+"."+dfstruct[mj].name])
    df=df.drop(structure_cols[i]);#Drop the Structure Column which is handled
 #Recalculate number of array & structure type column present in the DataFrame
  data_types=df.dtypes;
  array_cols=[];
  structure_cols=[];
  for c in data_types:
    if c[1][0:5] == "array":
      array_cols.append(c[0]);
    elif c[1][0:6] == "struct":
      structure_cols.append(c[0]);
 #Break the loop when there is no Complex Structure present
  if((len(array_cols)==0) & (len(structure_cols)==0)):
    temp=temp+1;

# **Check the Flatten Data**

In [ ]:
df.show()

+---------+-------------------+---------------------------+------------------------+--------------------------+---------------------------+-----------------+-------------------------+---------------------+------------------------+------------------+------------------+-----------------------+------------------------+------------------+-------------------+-----------------------------+--------------------+---------------------+-----------------------+-----------------------------------+--------------------------------+-------------------------------------+-------------------------------------------+----------------------------------------------+-------------------------------------+--------------------------------------------+--------------------------------------------+---------------------------------------+----------------------------------------+--------------------------------------+
|totalFeed|Expanded_feeds_code|Expanded_feeds_commentCount|Expanded_feeds_createdAt|Expanded_feeds_d

In [ ]:
df.count(),len(df.columns)

(296, 31)

In [ ]:
df.printSchema()

root
 |-- totalFeed: long (nullable = true)
 |-- Expanded_feeds_code: long (nullable = true)
 |-- Expanded_feeds_commentCount: long (nullable = true)
 |-- Expanded_feeds_createdAt: string (nullable = true)
 |-- Expanded_feeds_description: string (nullable = true)
 |-- Expanded_feeds_feedsComment: string (nullable = true)
 |-- Expanded_feeds_id: long (nullable = true)
 |-- Expanded_feeds_imagePaths: string (nullable = true)
 |-- Expanded_feeds_images: string (nullable = true)
 |-- Expanded_feeds_isdeleted: boolean (nullable = true)
 |-- Expanded_feeds_lat: long (nullable = true)
 |-- Expanded_feeds_lng: long (nullable = true)
 |-- Expanded_feeds_location: string (nullable = true)
 |-- Expanded_feeds_mediatype: long (nullable = true)
 |-- Expanded_feeds_msg: string (nullable = true)
 |-- Expanded_feeds_name: string (nullable = true)
 |-- Expanded_feeds_profilePicture: string (nullable = true)
 |-- Expanded_feeds_title: string (nullable = true)
 |-- Expanded_feeds_userId: long (nullable =